### YOLO-V3-Training on Custom Data
- Clone The Repository (Yolo v3 from https://github.com/ultralytics/yolov3)
- Update the architecture based of the application
- Load and Preprocess Data
- Arrange Data in Required Format
- Train The Model
- Model Evaluation

#### Import and update the the YOLOV3 code from github


In [2]:
# download the library ultralytics
!git clone https://github.com/ultralytics/yolov3

Cloning into 'yolov3'...


In [3]:
# go to folder yolov3
!cd yolov3

In [ ]:
%pip install -r requirements.txt

In [ ]:
# moving the uploaded file to desired location 
%mv /content/yolov3.cfg /content/yolov3/cfg/yolov3.cfg

### Loading and Cleaning Data
- load the images from the folder
- take the cv file and save individual files as .txt files

In [1]:
# import required libraries
import numpy as np
import pandas as pd
from PIL import Image

In [2]:
data_df = pd.read_csv("E:\\Work\\Gen2-Display\\Gen2_Icon_Detection_Dataset_08-23-22\\data.csv")
data_df.drop('Unnamed: 0', axis=1, inplace=True)
data_df["class"] = data_df['icon'].replace({"left_mirror_icon":0, "rcta_defect":1, "rcta_warning_icon":2, "reset_icon":3, "right_mirror_icon":4, "touch_control_icon":5})
data_df.head()

,icon,ymax,xmax,ymin,image_names,xmin,class
0,rcta_warning_icon,1252,1673,854,1.jpg,1221,2
1,left_mirror_icon,503,1310,105,1.jpg,858,0
2,left_mirror_icon,838,578,733,1.jpg,385,0
3,left_mirror_icon,1372,1231,1267,1.jpg,1038,0
4,touch_control_icon,937,1152,539,1.jpg,700,5


In [6]:
import os
os.makedirs("E:\\Work\\Gen2-Display\\Gen2_Icon_Detection_Dataset_08-23-22\\labels", exist_ok=True)

In [24]:
#load the data and save individual data into .txt files
dataset_path = "E:\\Work\\Gen2-Display\\Gen2_Icon_Detection_Dataset_08-23-22\\"
images_path = "E:\\Work\\Gen2-Display\\Gen2_Icon_Detection_Dataset_08-23-22\\images\\"
labels_path = "E:\\Work\\Gen2-Display\\Gen2_Icon_Detection_Dataset_08-23-22\\labels\\"
for name in data_df.image_names.unique():
    #take individual images
    image_df = data_df[data_df.image_names==name].copy()

    image_df['x_center'] = (image_df.xmax +image_df.xmin)/2
    image_df['y_center'] = (image_df.ymax +image_df.ymin)/2
    image_df['width'] = (image_df.xmax - image_df.xmin)
    image_df['height'] = (image_df.ymax - image_df.ymin)

    W,H = Image.open(images_path+name).size

    # normailze the bounding boxes
    image_df['x_center'] = image_df['x_center']/W
    image_df['y_center'] = image_df['y_center']/H
    image_df['width'] = image_df['width']/W
    image_df['height'] = image_df['height']/H


    image_df[['class', 'x_center', 'y_center', 'width', 'height']].to_csv(labels_path + name.split('.')[0] + '.txt', index=False, header=None, sep=" ")


In [25]:
# file containing train image names
iconDet_train = data_df.image_names.apply(lambda x: images_path + x).unique()
np.savetxt(f"{dataset_path}iconDet_train.txt", iconDet_train, fmt="%s")

In [26]:
# file containing validation names
iconDet_val = data_df.image_names.apply(lambda x: images_path + x).unique()
np.savetxt(f"{dataset_path}iconDet_val.txt", iconDet_val, fmt="%s")

In [27]:
with open(f'{dataset_path}iconDet.names', 'w') as f:
    f.write("left_mirror_icon\nrcta_defect\nrcta_warning_icon\nreset_icon\nright_mirror_icon\ntouch_control_icon")

In [28]:
with open(f'{dataset_path}iconDet.data', 'w') as f:
    f.write(f'classes=6\ntrain={dataset_path}iconDet_train.txt\nvalid={dataset_path}iconDet_val.txt\nnames={dataset_path}iconDet.names')

### Train the model

In [3]:
!python ./yolov3/train.py --weights "D:\Development\Deep_Learning\pytorch\01-Basics\weights\yolov3.pt" --cfg yolov3.cfg --data "E:\\Work\\Gen2-Display\\Gen2_Icon_Detection_Dataset_08-23-22\\iconDet.data" --img-size 640 --epochs 15

Apex recommended for faster mixed precision training: https://github.com/NVIDIA/apex
Namespace(adam=False, batch_size=16, bucket='', cache_images=False, cfg='.\\yolov3\\cfg\\yolov3.cfg', data='E:\\\\Work\\\\Gen2-Display\\\\Gen2_Icon_Detection_Dataset_08-23-22\\\\iconDet.data', device='', epochs=15, evolve=False, freeze_layers=False, img_size=[640], multi_scale=False, name='', nosave=False, notest=False, rect=False, resume=False, single_cls=False, weights='D:\\Development\\Deep_Learning\\pytorch\\01-Basics\\weights\\yolov3.pt')
Using CPU

Start Tensorboard with "tensorboard --logdir=runs", view at http://localhost:6006/
Model Summary: 222 layers, 6.15507e+07 parameters, 6.15507e+07 gradients
Optimizer groups: 75 .bias, 75 Conv2d.weight, 72 other


2022-08-23 17:24:34.415854: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-08-23 17:24:34.416177: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
c:\Users\F85SJ00\Anaconda3\envs\DL_PT\lib\site-packages\torch\functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Traceback (most recent call last):
  File "./yolov3/train.py", line 431, in <module>
    train(hyp)  # train normally
  File "./yolov3/train.py", line 119, in train
    ckpt = torch.load(weights, map_location=device)
  File "c:\Users\F85SJ00\Anaconda3\envs\DL_PT\lib\site-p